In [1]:
import numpy as np

# Example parameters
S0 = [100, 120]
sigma = [0.2, 0.25]
rho = np.array([[1.0, 0.1], [0.1, 1.0]])
w = [0.5, 0.5]
r = 0.05
T = 1.0
K = 110

def asian_basket_mc(S0, sigma, rho, w, r, T, K, option_type='call', N_steps=50, N_sim=2000, seed=42):
    """
    Asian basket option price using Monte Carlo simulation (2 assets).
    """
    np.random.seed(seed)
    
    S0 = np.array(S0)
    sigma = np.array(sigma)
    w = np.array(w)
    n_assets = 2
    dt = T / N_steps
    
    # Cholesky decomposition for correlation
    L = np.linalg.cholesky(rho)
    
    # Simulate paths
    payoffs = np.zeros(N_sim)
    for sim in range(N_sim):
        Z = np.random.randn(N_steps, n_assets)
        correlated_Z = Z @ L.T

        S = np.zeros((N_steps + 1, n_assets))
        S[0] = S0

        for t in range(1, N_steps + 1):
            S[t] = S[t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * correlated_Z[t-1])

        basket_avg = np.mean(S @ w) #creates S.w = a column vector of size [N_steps, 1]
        
        if option_type.lower() == 'call':
            payoffs[sim] = max(basket_avg - K, 0)
        else:
            payoffs[sim] = max(K - basket_avg, 0)
    
    price = np.exp(-r * T) * np.mean(payoffs)
    return price

price_mc = asian_basket_mc(S0, sigma, rho, w, r, T, K)
print(f"Monte Carlo Price (2 assets): {price_mc:.4f}")


Monte Carlo Price (2 assets): 5.7803


In [2]:
import numpy as np
from scipy.stats import norm

def asian_basket_three_moment(S0, sigma, rho, w, r, T, K, option_type='call'):
    """
    Asian basket option price using 3-moment matching (2 assets).
    """
    S0 = np.array(S0)
    sigma = np.array(sigma)
    w = np.array(w)
    
    # Covariance matrix
    cov_matrix = np.outer(sigma, sigma) * rho

    # Approximate moments of arithmetic average
    # Mean
    mu_i = S0 * np.exp(r * T)
    mu_A = np.sum(w * mu_i)
    
    # Variance
    var_matrix = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            cov_ij = cov_matrix[i, j]
            var_matrix[i, j] = S0[i] * S0[j] * (np.exp(cov_ij * T) - 1) * np.exp(2 * r * T)
    var_A = np.sum(w[:, None] * w[None, :] * var_matrix)
    
    # Skewness
    skew_matrix = np.zeros((2, 2, 2))
    for i in range(2):
        for j in range(2):
            for k in range(2):
                cov_ij = cov_matrix[i, j]
                cov_ik = cov_matrix[i, k]
                cov_jk = cov_matrix[j, k]
                skew_matrix[i, j, k] = (S0[i] * S0[j] * S0[k] *
                                         (np.exp(cov_ij + cov_ik + cov_jk) * T - 3*np.exp(cov_ij*T) + 2*np.exp(0)))
    skew_A = np.sum(w[:, None, None] * w[None, :, None] * w[None, None, :] * skew_matrix)

    # Approximate using a 3-moment lognormal (Cornish-Fisher approximation)
    # Compute adjusted d1, d2
    # Using simplified shifted lognormal approximation:
    sigma_adj = np.sqrt(np.log(var_A / mu_A**2 + 1))
    mu_adj = np.log(mu_A) - 0.5 * sigma_adj**2
    # Apply skew adjustment (Cornish-Fisher)
    z = (np.log(mu_A / K) - mu_adj) / sigma_adj
    skew_adj = skew_A / (var_A ** 1.5)
    z_cf = z + (1/6)*(z**2 - 1)*skew_adj

    if option_type.lower() == 'call':
        price = np.exp(-r*T) * (mu_A * norm.cdf(z_cf) - K * norm.cdf(z_cf - sigma_adj))
    else:
        price = np.exp(-r*T) * (K * norm.cdf(-(z_cf - sigma_adj)) - mu_A * norm.cdf(-z_cf))
    
    return price

# Example usage
S0 = [100, 120]
sigma = [0.2, 0.25]
rho = np.array([[1.0, 0.3], [0.3, 1.0]])
w = [0.5, 0.5]
r = 0.05
T = 1.0
K = 110

price_3mm = asian_basket_three_moment(S0, sigma, rho, w, r, T, K)
print(f"3-Moment Matching Price (2 assets): {price_3mm:.4f}")


3-Moment Matching Price (2 assets): 5.3648


In [2]:
import numpy as np
from scipy.stats import norm

# === Parameters ===
S0 = np.array([100, 95])          # Initial prices of the two assets
sigma = np.array([0.2, 0.25])     # Volatilities
rho = 0.6                         # Correlation between the two assets
r = 0.05                          # Risk-free rate
T = 2.0                           # Maturity
t_obs = np.array([0.5, 1.0, 1.5, 2.0])   # Observation dates
N = len(t_obs)
w = np.array([0.5, 0.5])          # Equal weights
K = 100                           # Strike

# === Correlation matrix ===
corr = np.array([[1.0, rho],
                 [rho, 1.0]])

# === Step 1: Compute first moment (mean) ===
m1 = 0.0
for t in t_obs:
    for i in range(2):
        m1 += w[i] * S0[i] * np.exp(r * t)
m1 /= N  # average over time

# === Step 2: Compute second moment (variance) ===
m2 = 0.0
for k in range(N):
    for l in range(N):
        for i in range(2):
            for j in range(2):
                cov_ij = S0[i] * S0[j] * np.exp(r * (t_obs[k] + t_obs[l])) * \
                         (np.exp(corr[i, j] * sigma[i] * sigma[j] * min(t_obs[k], t_obs[l])) - 1)
                m2 += w[i] * w[j] * cov_ij
m2 /= N**2  # time averaging
varA = m2

# === Step 3: Compute third central moment (approximate skewness) ===
m3 = 0.0
for k in range(N):
    for l in range(N):
        for m in range(N):
            for i in range(2):
                for j in range(2):
                    for p in range(2):
                        cov3 = S0[i]*S0[j]*S0[p]*np.exp(r*(t_obs[k]+t_obs[l]+t_obs[m])) * \
                               (np.exp((corr[i,j]*sigma[i]*sigma[j] +
                                        corr[i,p]*sigma[i]*sigma[p] +
                                        corr[j,p]*sigma[j]*sigma[p]) *
                                        min(t_obs[k], t_obs[l], t_obs[m])) - 1)
                        m3 += w[i]*w[j]*w[p]*cov3
m3 /= N**3
skewA = m3 / (varA ** 1.5)

# === Step 4: Fit a skewed lognormal using Cornish-Fisher expansion ===
# Variance-to-mean lognormal parameters
sigma_L = np.sqrt(np.log(1 + varA / (m1**2)))
mu_L = np.log(m1) - 0.5 * sigma_L**2

# Cornish-Fisher adjustment
d1 = (np.log(m1 / K) + 0.5 * sigma_L**2) / sigma_L
d2 = d1 - sigma_L

# Adjust for skewness
d1_CF = d1 + (skewA / 6) * (d1**2 - 1)
d2_CF = d2 + (skewA / 6) * (d2**2 - 1)

# === Step 5: Approximate price ===
price = np.exp(-r * T) * (m1 * norm.cdf(d1_CF) - K * norm.cdf(d2_CF))

# === Output ===
print("Asian Basket Option Pricing using Three-Moment Matching")
print(f"Mean (m1): {m1:.4f}")
print(f"Variance (m2): {varA:.4f}")
print(f"Skewness: {skewA:.4f}")
print(f"Approximate Call Price: {price:.4f}")


Asian Basket Option Pricing using Three-Moment Matching
Mean (m1): 103.8288
Variance (m2): 425.7749
Skewness: 13.0579
Approximate Call Price: 2.3841


In [3]:
import numpy as np

# === Parameters ===
S0 = np.array([100, 95])          # Initial prices of the two assets
sigma = np.array([0.2, 0.25])     # Volatilities
rho = 0.6                         # Correlation between the two assets
r = 0.05                          # Risk-free rate
T = 2.0                           # Maturity
t_obs = np.array([0.5, 1.0, 1.5, 2.0])   # Observation dates
N = len(t_obs)
w = np.array([0.5, 0.5])          # Equal weights
K = 100                           # Strike
n_paths = 200000                  # Monte Carlo paths
n_assets = 2                      # Two underlying assets

# === Step 1: Correlation matrix and Cholesky ===
corr = np.array([[1.0, rho],
                 [rho, 1.0]])
L = np.linalg.cholesky(corr)

# === Step 2: Simulate correlated GBM paths ===
dt_list = np.diff(np.insert(t_obs, 0, 0.0))  # time intervals between obs
S = np.zeros((n_paths, N, n_assets))
S[:, 0, :] = S0

for n in range(1, N):
    dt = dt_list[n]
    # Generate correlated random shocks
    Z = np.random.normal(size=(n_paths, n_assets))
    dW = Z @ L.T
    drift = (r - 0.5 * sigma**2) * dt
    diffusion = sigma * np.sqrt(dt) * dW
    S[:, n, :] = S[:, n-1, :] * np.exp(drift + diffusion)

# === Step 3: Compute basket value at each observation date ===
basket_values = np.dot(S, w)  # shape: (n_paths, N)

# === Step 4: Compute average basket price per path ===
basket_avg = np.mean(basket_values, axis=1)

# === Step 5: Compute discounted payoff ===
payoffs = np.exp(-r * T) * np.maximum(basket_avg - K, 0.0)

# === Step 6: Monte Carlo estimate ===
price_MC = np.mean(payoffs)
stderr_MC = np.std(payoffs) / np.sqrt(n_paths)

# === Output ===
print("Asian Basket Option Pricing using Monte Carlo Simulation")
print(f"Estimated Price: {price_MC:.4f}")
print(f"Standard Error: {stderr_MC:.4f}")


Asian Basket Option Pricing using Monte Carlo Simulation
Estimated Price: 5.4354
Standard Error: 0.0189


In [4]:
import numpy as np
from scipy.stats import norm

# === Parameters (same as MC) ===
S0 = np.array([100, 95])          # Initial prices
sigma = np.array([0.2, 0.25])     # Volatilities
rho = 0.6                         # Correlation
r = 0.05                          # Risk-free rate
T = 2.0                           # Maturity
t_obs = np.array([0.5, 1.0, 1.5, 2.0])  # Observation dates
N = len(t_obs)
w = np.array([0.5, 0.5])          # Equal weights
K = 100                           # Strike

# === Correlation matrix ===
corr = np.array([[1.0, rho],
                 [rho, 1.0]])

# === Step 1: Mean of average basket ===
m1 = 0.0
for t in t_obs:
    for i in range(2):
        m1 += w[i] * S0[i] * np.exp(r * t)
m1 /= N

# === Step 2: Variance of average basket ===
m2 = 0.0
for k in range(N):
    for l in range(N):
        for i in range(2):
            for j in range(2):
                cov_ij = S0[i]*S0[j]*np.exp(r*(t_obs[k]+t_obs[l])) * (
                    np.exp(corr[i, j]*sigma[i]*sigma[j]*min(t_obs[k], t_obs[l])) - 1
                )
                m2 += w[i]*w[j]*cov_ij
m2 /= N**2  # average over time pairs

varA = m2
stdA = np.sqrt(varA)

# === Step 3: Lognormal parameters ===
sigma_L = np.sqrt(np.log(1 + varA / (m1**2)))
mu_L = np.log(m1) - 0.5 * sigma_L**2

# === Step 4: Black–Scholes–type approximation ===
d1 = (np.log(m1 / K) + 0.5 * sigma_L**2) / sigma_L
d2 = d1 - sigma_L

price = np.exp(-r * T) * (m1 * norm.cdf(d1) - K * norm.cdf(d2))

# === Output ===
print("Asian Basket Option Pricing using Two-Moment Matching")
print(f"Mean (m1): {m1:.4f}")
print(f"Variance (m2): {varA:.4f}")
print(f"Approximate Call Price: {price:.4f}")


Asian Basket Option Pricing using Two-Moment Matching
Mean (m1): 103.8288
Variance (m2): 425.7749
Approximate Call Price: 9.0919


In [7]:
import numpy as np

def simulate_asian_basket_mc_time_dep(
    S0, vol_times_list, vol_vals_list, rho, w, r, t_obs, K,
    n_paths=200000, seed=42, option_type='call'
):
    """
    Monte Carlo for a 2-asset Asian basket with piecewise-constant vol term structures.
    - S0: array-like, shape (n_assets,)
    - vol_times_list: list of arrays; vol_times_list[i] are boundary times for asset i (including 0)
    - vol_vals_list: list of arrays; vol_vals_list[i] are vol values on intervals between vol_times (len = len(vol_times)-1)
    - rho: correlation matrix (n_assets x n_assets)
    - w: weights array, shape (n_assets,)
    - r: risk-free rate (assumed constant here; can be time-dependent if wanted)
    - t_obs: array of observation times (increasing)
    - K: strike
    - n_paths, seed: MC settings
    """
    np.random.seed(seed)
    S0 = np.array(S0)
    w = np.array(w)
    t_obs = np.array(t_obs)
    n_assets = len(S0)
    n_obs = len(t_obs)

    # Precompute integrated variances and covariances between observation times
    # For each asset i and interval (t_{k-1}, t_k], compute var_i(k) = ∫_{t_{k-1}}^{t_k} sigma_i(u)^2 du
    # For covariances between assets i,j over interval use ∫ sigma_i(u) sigma_j(u) du times rho_ij
    dt_intervals = np.diff(np.insert(t_obs, 0, 0.0))  # interval lengths where we advance from previous obs to next

    # Build per-interval sigma_i (value used on that interval) by mapping piecewise structure to t_obs intervals
    sigma_intervals = np.zeros((n_assets, n_obs))  # sigma_intervals[i,k] used on interval k (0..n_obs-1)
    for i in range(n_assets):
        times = vol_times_list[i]   # e.g. [0.0, 0.5, 1.0, 2.0]
        vals = vol_vals_list[i]     # len = len(times)-1
        # For each interval (previous_obs, obs_k], pick sigma value appropriate (we assume piecewise const aligned or we choose value on midpoint)
        for k in range(n_obs):
            t_left = 0.0 if k == 0 else t_obs[k-1]
            t_right = t_obs[k]
            # If piecewise constant but boundaries don't align exactly with t_left/t_right, we compute average sigma^2 over the interval:
            # we'll compute integral of sigma^2(u) on [t_left, t_right] by summing contributions on overlapping piecewise segments
            # but for simulation we only need sqrt of integral; we'll compute effective sigma * sqrt(dt) below.
            # To keep mapping simple, store the piecewise data and compute integrals directly in next step.
            sigma_intervals[i, k] = np.nan  # placeholder

    # Compute integrated covariances per interval: for interval k, compute
    # Var contribution for asset i: var_i_k = ∫_{t_{k-1}}^{t_k} sigma_i(u)^2 du
    var_interval = np.zeros((n_assets, n_obs))
    cov_interval = np.zeros((n_assets, n_assets, n_obs))
    for k in range(n_obs):
        a = 0.0 if k == 0 else t_obs[k-1]
        b = t_obs[k]
        for i in range(n_assets):
            # integrate sigma_i(u)^2 over [a,b]
            times = vol_times_list[i]
            vals = vol_vals_list[i]
            integral = 0.0
            for m in range(len(vals)):
                seg_left, seg_right = times[m], times[m+1]
                inter_left = max(a, seg_left)
                inter_right = min(b, seg_right)
                if inter_right > inter_left:
                    integral += vals[m]**2 * (inter_right - inter_left)
            var_interval[i, k] = integral
        for i in range(n_assets):
            for j in range(n_assets):
                # integrate sigma_i(u)*sigma_j(u) over [a,b]
                integral = 0.0
                # simple approach: iterate over partition given by union of times of i and j
                times_union = np.union1d(vol_times_list[i], vol_times_list[j])
                for m in range(len(times_union)-1):
                    seg_l, seg_r = times_union[m], times_union[m+1]
                    inter_left = max(a, seg_l)
                    inter_right = min(b, seg_r)
                    if inter_right > inter_left:
                        # find sigma_i on this subsegment
                        # find index of segment for asset i
                        idx_i = np.searchsorted(vol_times_list[i], (seg_l+seg_r)/2.0) - 1
                        idx_j = np.searchsorted(vol_times_list[j], (seg_l+seg_r)/2.0) - 1
                        sid = vol_vals_list[i][max(0, min(idx_i, len(vol_vals_list[i])-1))]
                        sjd = vol_vals_list[j][max(0, min(idx_j, len(vol_vals_list[j])-1))]
                        integral += sid * sjd * (inter_right - inter_left)
                cov_interval[i, j, k] = integral * (rho[i, j] if i != j else 1.0)

    # Now simulate: we can simulate increments of log S exactly per interval:
    # ln S_i(t_k) = ln S_i(t_{k-1}) + (r - 0.5 * (var_i_k/dt_k)) * dt_k + normal with variance var_i_k
    # But simpler: simulate correlated normals with covariance matrix Sigma_k where Sigma_k[i,i] = var_interval[i,k], Sigma_k[i,j] = cov_interval[i,j,k]
    # For each k generate multivariate normal N(0, Sigma_k)
    from numpy.linalg import cholesky

    # Precompute cholesky for each interval k
    chol_k = []
    for k in range(n_obs):
        Sigma_k = np.zeros((n_assets, n_assets))
        for i in range(n_assets):
            for j in range(n_assets):
                Sigma_k[i, j] = cov_interval[i, j, k]
        # numerical fix for PSD
        eps = 1e-12
        Sigma_k += np.eye(n_assets) * eps
        chol_k.append(np.linalg.cholesky(Sigma_k))

    # allocate S_paths at observation times
    S_paths = np.zeros((n_paths, n_obs, n_assets))
    S_paths[:, 0, :] = S0  # starting value at t0? We'll simulate increments from 0->t_obs[0] etc.
    # actually we need S at each observation time: do step-by-step
    lnS = np.log(np.tile(S0, (n_paths,1)))  # shape (n_paths, n_assets)

    for k in range(n_obs):
        Lk = chol_k[k]
        # draw n_paths multivariate normals: Z shape (n_paths, n_assets)
        Z = np.random.normal(size=(n_paths, n_assets))
        # correlated normals with cov Sigma_k: increments = Z @ Lk.T
        increments = Z @ Lk.T  # shape (n_paths, n_assets)
        # drift term for each asset i: r*dt - 0.5 * (var_interval[i,k])
        dt_k = (0.0 if k==0 else t_obs[k-1])  # not used; we only need integrated variance var_interval
        # For exact GBM log increment: ln S(t_k) = ln S(t_{k-1}) + int r dt - 0.5 * var_i_k + increment_i
        # assume constant r -> int_0^{delta} r du = r*(b-a)
        a = 0.0 if k==0 else t_obs[k-1]
        b = t_obs[k]
        int_r = r * (b - a)
        drift = int_r - 0.5 * var_interval[:, k]  # shape (n_assets,)
        # apply increment to all paths
        lnS = lnS + drift + increments
        S_now = np.exp(lnS)
        S_paths[:, k, :] = S_now

    # basket values and payoff
    basket_vals = S_paths @ w  # (n_paths, n_obs)
    basket_avg = basket_vals.mean(axis=1)
    T = t_obs[-1]
    payoffs = np.exp(-r * T) * np.maximum(basket_avg - K, 0.0) if option_type == 'call' else np.exp(-r*T)*np.maximum(K - basket_avg, 0.0)
    price = payoffs.mean()
    stderr = payoffs.std(ddof=1)/np.sqrt(n_paths)
    return price, stderr


In [8]:
import numpy as np
from scipy.stats import norm

def two_moment_matching_time_dep(S0, vol_times_list, vol_vals_list, rho, w, r, t_obs, K):
    """
    Compute 2-moment matching price for basket average with time-dependent vols.
    Inputs similar to MC function.
    """
    S0 = np.array(S0)
    w = np.array(w)
    t_obs = np.array(t_obs)
    n_assets = len(S0)
    N = len(t_obs)
    # Precompute integrals C_ij(tk, tl)
    # We'll compute for all pairs (k,l)
    C = np.zeros((n_assets, n_assets, N, N))
    for k in range(N):
        for l in range(N):
            tmin = min(t_obs[k], t_obs[l])
            a = 0.0
            b = tmin
            for i in range(n_assets):
                for j in range(n_assets):
                    # integrate sigma_i(u)*sigma_j(u) over [a,b]
                    integral = 0.0
                    times_union = np.union1d(vol_times_list[i], vol_times_list[j])
                    for m in range(len(times_union)-1):
                        seg_l = times_union[m]; seg_r = times_union[m+1]
                        inter_left = max(a, seg_l)
                        inter_right = min(b, seg_r)
                        if inter_right > inter_left:
                            # pick sigma_i on midpoint
                            idx_i = np.searchsorted(vol_times_list[i], 0.5*(seg_l+seg_r)) - 1
                            idx_j = np.searchsorted(vol_times_list[j], 0.5*(seg_l+seg_r)) - 1
                            si = vol_vals_list[i][max(0, min(idx_i, len(vol_vals_list[i])-1))]
                            sj = vol_vals_list[j][max(0, min(idx_j, len(vol_vals_list[j])-1))]
                            integral += si * sj * (inter_right - inter_left)
                    C[i,j,k,l] = rho[i,j] * integral

    # Compute m1
    m1 = 0.0
    for k in range(N):
        for i in range(n_assets):
            m1 += w[i] * S0[i] * np.exp(r * t_obs[k])
    m1 /= N

    # Compute Var(A)
    varA = 0.0
    for k in range(N):
        for l in range(N):
            for i in range(n_assets):
                for j in range(n_assets):
                    cov_ij = S0[i]*S0[j]*np.exp(r*(t_obs[k]+t_obs[l])) * (np.exp(C[i,j,k,l]) - 1.0)
                    varA += w[i]*w[j]*cov_ij
    varA /= N**2

    # Match lognormal
    sigma_A = np.sqrt(np.log(1 + varA / (m1**2)))
    mu_A = np.log(m1) - 0.5 * sigma_A**2
    d1 = (np.log(m1 / K) + 0.5 * sigma_A**2) / sigma_A
    d2 = d1 - sigma_A
    price = np.exp(-r * t_obs[-1]) * (m1 * norm.cdf(d1) - K * norm.cdf(d2))
    return price, m1, varA


In [9]:
# Example setup (2 assets)
S0 = [100, 95]
# Suppose these are implied ATM vols for total maturities [0.5, 1.0, 1.5, 2.0]
imp_mats = np.array([0.5, 1.0, 1.5, 2.0])
imp_vols_asset1 = np.array([0.18, 0.19, 0.195, 0.2])
imp_vols_asset2 = np.array([0.22, 0.225, 0.23, 0.235])

# Build piecewise instantaneous vol per asset via forward variance decomposition
def piecewise_instantaneous_from_imp(imp_times, imp_vols):
    # imp_vols are Black implied vol (annualized) at imp_times
    V = imp_vols**2 * imp_times  # cumulative variance at each maturity
    times = np.concatenate(([0.0], imp_times))
    vals = []
    for k in range(len(imp_times)):
        dt = imp_times[k] - (imp_times[k-1] if k>0 else 0.0)
        forward_var = (V[k] - (V[k-1] if k>0 else 0.0)) / dt
        vals.append(np.sqrt(max(forward_var, 0.0)))  # instantaneous vol on (t_{k-1}, t_k]
    return times, np.array(vals)

vol_times_list = []
vol_vals_list = []
for impv in [imp_vols_asset1, imp_vols_asset2]:
    times, vals = piecewise_instantaneous_from_imp(imp_mats, impv)
    vol_times_list.append(times)
    vol_vals_list.append(vals)

rho = np.array([[1.0, 0.6],[0.6,1.0]])
w = [0.5,0.5]
r = 0.05
t_obs = np.array([0.5,1.0,1.5,2.0])
K = 100

# Two moment price
price_2mm, m1, varA = two_moment_matching_time_dep(S0, vol_times_list, vol_vals_list, rho, w, r, t_obs, K)
print("Two-moment matching price:", price_2mm)

# MC price
price_mc, stderr = simulate_asian_basket_mc_time_dep(S0, vol_times_list, vol_vals_list, rho, w, r, t_obs, K, n_paths=200000)
print("MC price:", price_mc, "stderr:", stderr)


Two-moment matching price: 8.584954764838338
MC price: 8.518949279462854 stderr: 0.027726592386275246
